In [26]:
from random import uniform
import numpy as np
import random
from sklearn import svm
import math

def count_diff(list_1, list_2):
    count = 0
    for x in range(0, len(list_1)):
        if list_1[x] != list_2[x]:
            count += 1
    return count 

def sign(x):
    if x > 0:
        return 1
    elif x < 0:
        return -1
    else:
        return 0
    
def reclassify(w, class_list, points):
        for i in range(0, len(class_list)):
            class_list[i] = sign(np.dot(points[i], w))

trials = 1000
N = 100
sup_vector_avg = 0
percent = 0

for j in range(0, trials):
    d = 2
    true_class = [0]*N

#PLA PLA PLA PLA PLA
    while true_class == [0]*N or true_class == [1]*N or true_class == [-1]*N:
        x_1 = [uniform(-1, 1) for x in range(0, 2)]
        y_1 = [uniform(-1, 1) for x in range(0,2)]
        f_fit = np.polyfit(x_1, y_1, 1)

        x_n = [uniform(-1, 1) for x in range(0, N)]
        y_n = [uniform(-1, 1) for x in range(0, N)]

        for i in range(0, N):
            if y_n[i] > (y_n[i]*f_fit[0] + f_fit[1]):
                true_class[i] = 1
            else:
                true_class[i] = -1

#WEIGHT CALCULATION
    w = [0,0,0] # g(x) 

    classification = [0]*N
    points = [[1, x_n[i], y_n[i]] for i in range(0, N)]

    while true_class != classification:
        wrong = []
        for i in range(0, N):
            if classification[i] != true_class[i]:
                wrong.append(points[i])
        point = random.choice(wrong)
        idx = points.index(point)
        temp_sign = true_class[idx]
        w = [w[0] + temp_sign*point[0], w[1] + temp_sign*point[1], w[2] + temp_sign*point[2]]
        reclassify(w, classification, points)

#ITERATION TRACKING AND SVM   
    if j%100 == 0:
        print(j)
        
    clf = svm.SVC(kernel="linear", C=math.inf)
    clf.fit(points, true_class)
    
    sup_vector_avg += len(clf.support_vectors_)/trials
    
    x_large = [[1, uniform(-1, 1), uniform(-1, 1)] for i in range(0, 1000)]
    y_f = [(x_large[i][1]*f_fit[0] + f_fit[1]) for i in range(len(x_large))]

    correct_class = [sign(x_large[i][2] - y_f[i]) for i in range(0, len(x_large))]

    PLA_error = [sign(np.dot(x_large[i], w)) for i in range(0, 1000)]

    clf_guess = clf.predict(x_large)
    
    diff_svm = count_diff(clf_guess, correct_class)
    diff_PLA = count_diff(correct_class, PLA_error)
    
    E_out_PLA = diff_PLA/1000
    E_out_svm = diff_svm/1000
    
    if (E_out_svm < E_out_PLA):
        percent += 1/trials

print(percent)
print(sup_vector_avg)

0
100
200
300
400
500
600
700
800
900
0.45600000000000035
2.9970000000000026
